In [74]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import dash
from dash import Dash, dcc, html, Input, Output
import matplotlib.pyplot as plt
import plotly.express as px
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import plotly.graph_objects as go
from io import BytesIO
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error


# Set download directory for Excel files
download_dir = os.getcwd()

# Set Chrome options for automatic downloads
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,  # Change default download directory
    "download.prompt_for_download": False,       # Disable download prompts
    "directory_upgrade": False,                   # Automatically overwrite files
}
options.add_experimental_option("prefs", prefs)


options.add_argument("--headless")  # Optional: run in headless mode
driver = webdriver.Chrome(options=options)

# Launch browser
driver = webdriver.Chrome(options=options)

# Open Tableau page
tableau_url = "https://fragilestatesindex.org/global-data/"

driver.get(tableau_url)
# Wait for the page to load (adjust based on your Tableau dashboard)
time.sleep(10)
 
iframe = driver.find_element(By.TAG_NAME, "iframe")
driver.switch_to.frame(iframe)

# Locate the download button (update with actual locator of your download button)
download_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='download']/span[2]"))
)

# Click the download button
download_button.click()


select_option = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='viz-viewer-toolbar-download-menu']/div[3]/div/div/span[2]"))
)
select_option.click()

download_excel = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='export-crosstab-options-dialog-Dialog-BodyWrapper-Dialog-Body-Id']/div/div[3]/button"))
)
download_excel.click()

# Wait for download to complete (adjust time based on file size and connection speed)
time.sleep(15)

# Verify downloaded file
downloaded_files = os.listdir(download_dir)
excel_files = [f for f in downloaded_files if f.endswith("Rankings.xlsx")]

if excel_files:
    print(f"Downloaded Excel file: {excel_files[0]}")
else:
    print("No Excel file found in the download directory.")

# Close the browser
driver.quit()

Downloaded Excel file: Rankings.xlsx


In [75]:

"""
Right now we are downloading data from 
https://fragilestatesindex.org/
Then processing it for move up to red
We need to automate this process for use in our new green to red pipeline(WIP)
On a monthly basis we need to do a data pull and get any new data
If we put it in a dash app we can use POSIT CONNECT to keep the script persisting
"""
df = pd.read_excel("Rankings.xlsx")

cols = []
for col in df.columns:
    if "Unnamed" in col:
        cols.append(df[col].iloc[0])
    else:
        cols.append(col)
        
df.columns = cols
cell_value = int(df.iloc[0,3])

df = df[1:]
df = df[["Country", "Total"]]
df.columns = ["Country", "Total"]
df["Year"] = cell_value

In [76]:
df

,Country,Total,Year
1,Somalia,111.3,2024
2,Sudan,109.3,2024
3,South Sudan,109.0,2024
4,Syria,108.1,2024
5,Congo Democratic Republic,106.7,2024
...,...,...,...
175,Denmark,15.9,2024
176,New Zealand,15.9,2024
177,Iceland,15.2,2024
178,Finland,14.3,2024


In [77]:
df.to_excel("FSI_output.xlsx")

In [78]:


# Path to your ChromeDriver
# Set up the Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Optional: run in headless mode
driver = webdriver.Chrome(options=options)

url = "https://fragilestatesindex.org/excel/"
driver.get(url)

# Wait for the page to load (adjust time if necessary)
driver.implicitly_wait(10)

# Extract links
excel_links = set()
for link in driver.find_elements(By.TAG_NAME, "a"):
    href = link.get_attribute("href")
    if href and (".xlsx" in href or ".xls" in href):
        excel_links.add(href)

print("Excel Links Found:", excel_links)
driver.quit()

Excel Links Found: {'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2016.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2013.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2019/04/fsi-2019.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2015.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2008.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2020/05/fsi-2020.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2017.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2018/04/fsi-2018.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2023/06/FSI-2023-DOWNLOAD.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/2022/07/fsi-2022-download.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2012.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2009.xlsx', 'https://fragilestatesindex.org/wp-content/uploads/data/fsi-2010.xlsx', 'https://fr

In [79]:
scraped_df = pd.DataFrame()

# Define headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

columns_to_keep = ["Country", "Year", "Total"]

# Loop through each link, download the file, and read it
for link in excel_links:
    try:
        # Send GET request with headers
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Read the Excel file into a DataFrame
        df = pd.read_excel(BytesIO(response.content), usecols=columns_to_keep)
        
        # Append the data to the combined DataFrame
        scraped_df = pd.concat([scraped_df, df], ignore_index=True)
    except Exception as e:
        print(f"Failed to process {link}: {e}")

#Modify Year to represent an Integer
scraped_df['Year'] = scraped_df['Year'].astype(str).str[:4].astype(int)

#Read FSI output that contains data for 2004
df1 = pd.read_excel("FSI_output.xlsx")
df1_filtered = df1[columns_to_keep]

#Concat scraped dataframe and data read from Rankings
df_combined = pd.concat([scraped_df, df1_filtered], ignore_index=True).reset_index(drop=True)

# Remove leading/trailing spaces and make them titles
df_combined['Country'] = df_combined['Country'].str.strip()  
df_combined['Country'] = df_combined['Country'].str.title()
# Sort data by year
df_combined = df_combined.sort_values(by='Year')


#Write into a new xlsx
df_combined.to_excel("FSI_Combined.xlsx")

# Display the combined DataFrame
df_combined

,Country,Year,Total
2777,South Africa,2006,55.7
2778,Estonia,2006,51.0
2779,Slovak Republic,2006,49.9
2780,Lithuania,2006,49.7
2733,Zambia,2006,79.6
...,...,...,...
3301,Mongolia,2024,50.7
3318,Qatar,2024,39.8
3348,Norway,2024,12.7
3320,Mauritius,2024,37.8


In [80]:
app = dash.Dash(__name__)

app.layout = html.Div([

    html.Label("Select Country:"),
    dcc.Dropdown(
        id='country-dropdown',
        options=sorted(
            [{'label': country, 'value': country} for country in df_combined['Country'].unique()],
            key=lambda x: x['label']),
        value='United States',  # Default value
        placeholder="Select a country"
    ),
    
    html.Div([
        dcc.Graph(id='line-chart'),
        dcc.Graph(id='country-predictions'),
        dcc.Graph(id='country-graph')
    ])
])

# Callback to update the line chart based on selected country
@app.callback(
    Output('line-chart', 'figure'),
    Input('country-dropdown', 'value')
)
def update_chart(selected_country):
    # Filter data by selected country
    filtered_df = df_combined[df_combined['Country'] == selected_country]

    # Create the line chart
    line_chart = px.line(filtered_df, x='Year', y='Total', title=f'Total by Year for {selected_country}')
    return line_chart



In [81]:
# Callback to update the bar graph based on selected country
@app.callback(
    Output('country-graph', 'figure'),
    Input('country-dropdown', 'value')
)
def update_graph(selected_country):

    df = pd.read_excel("FSI_Combined.xlsx")
    
    # Filter data for the selected country
    country_data = df[df['Country'] == selected_country].sort_values('Year')
    
    # Extract years and totals
    years = country_data['Year'].values
    totals = country_data['Total'].values
    
    # Calculate yearly changes
    yearly_changes = [totals[i] - totals[i-1] for i in range(1, len(totals))]
    change_years = years[1:]  # Years corresponding to changes
    
    # Create bar colors: green for negative (improvement), red for positive
    bar_colors = ['green' if change < 0 else 'red' for change in yearly_changes]
    
    # Create the figure
    bar_graph = go.Figure()
    
    # Add bars for yearly changes
    bar_graph.add_trace(go.Bar(
        x=change_years,
        y=yearly_changes,
        marker_color=bar_colors,
        name='Yearly Changes'
    ))
    
    # Update layout
    bar_graph.update_layout(
        title=f"Net Change in Totals for {selected_country}",
        xaxis_title="Year",
        yaxis_title="Change in Total",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        template='plotly_white'
    )
    
    return bar_graph

# Callback to update the bar graph based on selected country
@app.callback(
    Output('country-predictions', 'figure'),
    Input('country-dropdown', 'value')
)
def update_graph(selected_country):
    # Load data
    file_path = "FSI_Combined.xlsx"  # Replace with your file path
    data = pd.read_excel(file_path)
    # Filter data for the selected country
    country_data = data[data['Country'] == selected_country].sort_values('Year')
    
    # Ensure all years are present
    all_years = pd.DataFrame({'Year': range(2006, 2025)})
    country_data = pd.merge(all_years, country_data, on='Year', how='left')
    
    # Train-test split
    train_years = range(2006, 2021)
    test_years = range(2021, 2025)
    train_data = country_data[country_data['Year'].isin(train_years)]
    test_data = country_data[country_data['Year'].isin(test_years)]
    
    # Check for sufficient data
    if train_data.empty or len(train_data) < 2:
        return go.Figure().update_layout(
            title=f"No sufficient data for {selected_country}",
            xaxis_title="Year",
            yaxis_title="Total Score"
        )
    
    # Prepare features and targets
    X_train = train_data[['Year']].values
    y_train = train_data['Total'].values
    X_test = test_data[['Year']].values
    y_test = test_data['Total'].values
    
    # Train linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test) if len(X_test) > 0 else []
    
    # Create the figure
    fig = go.Figure()
    # Add training data
    fig.add_trace(go.Scatter(
        x=train_data['Year'], y=y_train,
        mode='markers', name='Training Data'
    ))
    # Add test data
    fig.add_trace(go.Scatter(
        x=test_data['Year'], y=y_test,
        mode='markers', name='Actual Test Data'
    ))
    # Add predictions
    if predictions is not None and predictions.size > 0:
        fig.add_trace(go.Scatter(
            x=test_data['Year'], y=predictions,
            mode='lines+markers', name='Predictions'
        ))
    
    # Update layout
    fig.update_layout(
        title=f"Linear Regression Predictions for {selected_country}",
        xaxis_title="Year",
        yaxis_title="Total Score",
        legend_title="Legend"
    )
    
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

